In [1]:
# !wget http://vision.cs.utexas.edu/projects/finegrained/utzap50k/ut-zap50k-images-square.zip!wget http://vision.cs.utexas.edu/projects/finegrained/utzap50k/ut-zap50k-data.zip
# !unzip ut-zap50k-images-square.zip
# !unzip ut-zap50k-data.zip

#Importing the required libraries

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib as mpl
%matplotlib inline
#from tqdm import tqdm_notebook
import os
from glob import glob
from os.path import isdir,abspath,join
from os import listdir


#import numpy as np
from tensorflow.python.keras import backend as K
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import load_img,img_to_array
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.applications import imagenet_utils
from tensorflow.keras.callbacks import ModelCheckpoint,LearningRateScheduler,EarlyStopping
from tensorflow.keras.optimizers import Adam


In [3]:

from keras.utils import np_utils

# from keras.preprocessing import image
# from keras.applications.vgg16 import VGG16, preprocess_input
#from keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
# from keras.applications import imagenet_utils
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix
import itertools


import warnings
warnings.filterwarnings('ignore')

import seaborn as sns

Using TensorFlow backend.


In [4]:
#!pip install mat4py   #For reading the .mat files in the dataset
import mat4py

In [5]:
image_paths = mat4py.loadmat("ut-zap50k-data/image-path.mat")
image_path=[i[0] for i in image_paths["imagepath"]]
meta_data=pd.read_csv("ut-zap50k-data/meta-data.csv")
meta_data["path"]=image_path

In [6]:
#fgrational = mat4py.loadmat("ut-zap50k-feats/zappos-gist.ma

In [7]:
categories=pd.read_csv("ut-zap50k-data/meta-data-bin.csv")
#meta_data2["path"]=image_path2

In [8]:
categories.tail()

,CID,Category.Shoes,Category.Boots,Category.Sandals,Category.Slippers,SubCategory.Oxfords,SubCategory.Mid.Calf,SubCategory.Heel,SubCategory.Ankle,SubCategory.Boot,...,ToeStyle.Center Seam,ToeStyle.Pointed Toe,ToeStyle.Algonquin,ToeStyle.Almond,ToeStyle.Bump Toe,ToeStyle.Apron Toe,ToeStyle.Wide Toe Box,ToeStyle.Snip Toe,ToeStyle.Peep Toe,ToeStyle.Medallion
50020,8168300-128,1,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
50021,8168300-742,1,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
50022,8168300-57113,1,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
50023,8168300-384413,1,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
50024,8169712-2557,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
#categories['CID'].iloc[0:10]
meta_data.tail()

,CID,Category,SubCategory,HeelHeight,Insole,Closure,Gender,Material,ToeStyle,path
50020,8168300-128,Shoes,Heels,2in - 2 3/4in,Padded,Slip-On,Women,Leather,Capped Toe;Pointed Toe;Closed Toe,Shoes/Heels/J. Renee/8168300.128.jpg
50021,8168300-742,Shoes,Heels,2in - 2 3/4in,Padded,Slip-On,Women,Leather,Capped Toe;Pointed Toe;Closed Toe,Shoes/Heels/J. Renee/8168300.742.jpg
50022,8168300-57113,Shoes,Heels,2in - 2 3/4in,Padded,Slip-On,Women,Leather,Capped Toe;Pointed Toe;Closed Toe,Shoes/Heels/J. Renee/8168300.57113.jpg
50023,8168300-384413,Shoes,Heels,2in - 2 3/4in,Padded,Slip-On,Women,Leather,Capped Toe;Pointed Toe;Closed Toe,Shoes/Heels/J. Renee/8168300.384413.jpg
50024,8169712-2557,Shoes,Sneakers and Athletic Shoes,NaN,NaN,Lace up,Men,Leather,Round Toe,Shoes/Sneakers and Athletic Shoes/Travis Mathe...


In [10]:
#isSneaker = (meta_data['SubCategory'] == 'Sneakers and Athletic Shoes')
#lets start by just using sneakers
#df = meta_data[isSneaker]
df = pd.merge(meta_data, categories,  how='left', on='CID')# left_on=['CID'], right_on = ['CID'])

df.shape


# make column for each Boots, Sandals, Shoes, Slippers, and BRAND
#df = meta_data
df['path'].head()

0     Shoes/Oxfords/Bostonian/100627.72.jpg
1    Shoes/Oxfords/Bostonian/100627.255.jpg
2     Shoes/Oxfords/Bostonian/100657.72.jpg
3    Shoes/Oxfords/Bostonian/100657.216.jpg
4       Boots/Mid-Calf/Durango/101026.3.jpg
Name: path, dtype: object

In [11]:
df_clean=df
df_clean.loc[df_clean.path.str.contains("./",regex=False),"path"] = [i.replace("./","%2E/") for i in df.loc[df_clean.path.str.contains("./",regex=False),"path"]]
df=df_clean


In [12]:
#x = os.walk(df['path'][0])
df['path_and_file'] = df.path.apply(lambda path: (os.path.normpath(path)).split(os.sep) ) 

In [13]:
df_to_add = pd.DataFrame(df['path_and_file'].tolist(), columns=['Category1','Category2','Brand','Filename'])
df = df.merge(df_to_add, left_index=True, right_index=True)

In [14]:
from datetime import datetime
today = datetime.today()

# this is where our cleaned up file will put put
summary_file = f'./summary_{today:%b-%d-%Y}.pkl'
print(summary_file)

df.to_pickle(summary_file)

summary_file = f'./summary_{today:%b-%d-%Y}.json'
df.to_json(summary_file)


./summary_Feb-02-2020.pkl


In [15]:
summary_file

'./summary_Feb-02-2020.json'

In [16]:

#output = df['path'].str.replace(paths, dst)


# COMBINE meta_data and categories.  then iterate over all files to get vector

In [17]:
print(os.getcwd())

/home/ergonyc/Projects/Insight/UTzappos50k


In [18]:
# eventually move this into a module
class ImageFeatures: 
    """Process image files and extract features
    Attributes: 
        model (obj): model used to extract features
        image_vector (array): flattened and normalized array of image RGB codes
        img_feature (array): Pooling layer of model 
        
    
    """
    def __init__(self, img_path, pic_width, pic_height, model):
        
        self.model = model
        img_vec = self.image_process(img_path, pic_width, pic_height)
        self.feature_extraction(img_vec, model)
            
    def image_process(self, img_path, pic_width, pic_height):
        """Get flattened RGB image vector"""

        img_data = image.load_img(img_path, target_size=(pic_width, pic_height))
        image_vector = image.img_to_array(img_data)
        return(image_vector)

    def feature_extraction(self, img_vector, model):
        """Extract image feature vector"""
        img_d = img_vector.copy() #Absolutely essential, otherwise preprocess call will overwrite img_vector
        img_d = np.expand_dims(img_d, axis=0)
        img_d = preprocess_input(img_d) #Problem here, must be convention of keras to pass by reference?
        img_d = imagenet_utils.preprocess_input(img_d)
        self.img_features = model.predict(img_d)

        

In [19]:
#######Begin trying to come up with model to tag posts in database#######
from os import listdir
from os.path import isfile, join
from sklearn.decomposition import PCA


def process_image_files(path, label, model): 
    """ Extract image features from image files stored in path
    
    Args: 
        path (str): path to image location 
        label (int): image classification label -- we'll just use 
        model (obj): Keras model for feature extraction 
        
    Return: 
        df (dataframe): pandas dataframe containing filepath, filename labels and image fx
    """
    
    img_height = 224
    img_width = 224
    
    #get all file names in directory, get full file path and assign lables 
    file_names = [f for f in listdir(path) if (isfile(join(path, f)) and f !='.DS_Store')]
    full_file_path = [(path + file) for file in file_names]
    labels = np.ones(len(file_names), dtype = int)*label
    
    #instanstiate ImageFeatures object and save to array 
    image_obj = []
    for file in tqdm_notebook(full_file_path): 
        image_obj.append(ImageFeatures(file, img_height, img_width, model))
    
    #extract image features 
    image_features = []
    for obj in image_obj: 
        image_features.append(obj.img_features)
        
    
    #store all relevant info in a pandas datafram 
    df = pd.DataFrame()
    df['labels'] = labels 
    df['image_features'] = image_features
    df['image_file_path'] = full_file_path
    df['file_names'] = file_names
    
    return(df)
        

In [20]:
from tqdm import tqdm_notebook        
        
def add_features_to_database(json_filepath, image_filepath, cnn_model): 
    ''' function to take in web-scraped data from etsy and artsy 
    and returns a uniform pandas dataframe with relevant features
    
    updated database saved to json_filepath directory
    
    Args: 
    image_filepath (str): path to folder containing images
    
    Returns: 
    new_df (df): updated pandas dataframe containing image_features, image_quality_score
    
    '''
    #initialize image size
    
    pwidth = 224
    pheight = 224 #
    
    #Load in data  (pass names to json DBs around rather than DataFrames)
    json_file = json.load(open(json_filepath))
    image_df = pd.DataFrame(json_file)

    
    #Convert images to RGB arrays and extract features
    img_objects = []
    for rel_path in tqdm_notebook(image_df['path']):
        print(rel_path)
        print(type(rel_path))
        img_path = image_filepath + rel_path #[0]['path']
        img_obj = ImageFeatures(img_path, pwidth, pheight, cnn_model)
        img_objects.append(img_obj)
        
    #loop over img_objects to get array of features and predict quality scores
    cnn_features = []
    #quality_labels = []
    for entry in img_objects: 
        cnn_features.append(entry.img_features[0,:])
        #quality_labels.append(quality_model.predict(entry.img_features)[0])
    
    #Create new dataframe with relevant features
    new_df = pd.DataFrame()
    #new_df['image_url'] = image_df['image_urls']
    new_df['image_features'] = cnn_features
    #new_df['image_quality_score'] = quality_labels
    #new_df['price'] = [get_price(price) for price in image_df['price']]
    #new_df['vendor'] = [vendor] * len(quality_labels)
           
    save_filepath = json_filepath.split('.json')[0] + '_processed.json'
    new_df.to_json(save_filepath, orient = 'records')
    
    return(new_df)

In [21]:
# need to enforce that we are in the root directory.
#os.chdir('..')
#os.chdir('sneaks')
os.chdir('/home/ergonyc/Projects/Insight/UTzappos50k')

image_df = df


# #from utils import train_val_generator
data_path = 'sneaks/data' # path of the data
train_path = 'sneaks/train'
test_path = 'sneaks/test'

TARGET_SZ = 224

input_shape = (TARGET_SZ,TARGET_SZ,3) #(img_width,i mg_height,img_channel)')
#batch_size = 32

cnn_model = MobileNetV2(weights = 'imagenet', 
                            include_top = False, 
                            input_shape = input_shape, 
                            pooling = 'avg')

model_name = 'MobileNetV2'
# with graph.as_default():
#     img_features = network_model.predict(img_vector)
cnn_model.

In [ ]:

pwidth = TARGET_SZ
pheight = TARGET_SZ #
json_filepath = summary_file
#Load in data  (pass names to json DBs around rather than DataFrames)
json_file = json.load(open(json_filepath))
image_df = pd.DataFrame(json_file)

image_filepath = data_path

#Convert images to RGB arrays and extract features
img_objects = []
for rel_path in tqdm_notebook(image_df['path']):
    #print(rel_path)

    img_path = join(image_filepath, rel_path) #[0]['path']
    img_obj = ImageFeatures(img_path, pwidth, pheight, cnn_model)
    img_objects.append(img_obj)


Shoes/Oxfords/Bostonian/100627.72.jpg
Shoes/Oxfords/Bostonian/100627.255.jpg
Shoes/Oxfords/Bostonian/100657.72.jpg
Shoes/Oxfords/Bostonian/100657.216.jpg
Boots/Mid-Calf/Durango/101026.3.jpg
Sandals/Heel/Finn Comfort/101093.316189.jpg
Sandals/Heel/Finn Comfort/101093.316195.jpg
Sandals/Heel/Finn Comfort/101093.316313.jpg
Sandals/Heel/Finn Comfort/101093.342648.jpg
Boots/Mid-Calf/Frye/101175.325.jpg
Boots/Mid-Calf/Frye/101175.598.jpg
Boots/Ankle/Frye/101178.68022.jpg
Boots/Ankle/Frye/101178.238499.jpg
Boots/Ankle/Frye/101182.3.jpg
Boots/Ankle/Frye/101182.381.jpg
Boots/Mid-Calf/Frye/101183.3.jpg
Boots/Mid-Calf/Frye/101183.381.jpg
Boots/Mid-Calf/Frye/101183.170618.jpg
Boots/Mid-Calf/Frye/101183.246719.jpg
Boots/Ankle/Frye/101186.3.jpg
Boots/Mid-Calf/Frye/101191.3.jpg
Boots/Mid-Calf/Frye/101191.20.jpg
Boots/Mid-Calf/Frye/101191.598.jpg
Boots/Mid-Calf/Frye/101191.24883.jpg
Boots/Mid-Calf/Frye/101191.65553.jpg
Boots/Mid-Calf/Frye/101191.81355.jpg
Boots/Mid-Calf/Frye/101191.238499.jpg
Boots/Mi

Shoes/Oxfords/Rockport/106639.624.jpg
Shoes/Oxfords/Rockport/106639.1177.jpg
Shoes/Sneakers and Athletic Shoes/New Balance/106667.751.jpg
Shoes/Oxfords/Deer Stags/106701.3.jpg
Shoes/Oxfords/Deer Stags/106701.6.jpg
Shoes/Oxfords/Deer Stags/106702.3.jpg
Shoes/Oxfords/Deer Stags/106703.3.jpg
Boots/Mid-Calf/UGG/107213.3.jpg
Boots/Mid-Calf/UGG/107213.18.jpg
Boots/Mid-Calf/UGG/107213.278.jpg
Boots/Mid-Calf/UGG/107213.401.jpg
Boots/Mid-Calf/UGG/107213.621.jpg
Slippers/Slipper Flats/UGG/107219.33900.jpg
Slippers/Slipper Flats/UGG/107219.152359.jpg
Slippers/Slipper Flats/UGG/107219.152360.jpg
Slippers/Slipper Flats/UGG/107219.152374.jpg
Boots/Mid-Calf/UGG/107225.18.jpg
Shoes/Loafers/Franco Sarto/107227.8874.jpg
Boots/Ankle/Caterpillar/107452.3.jpg
Boots/Ankle/Caterpillar/107452.20.jpg
Boots/Ankle/Caterpillar/107452.328.jpg
Boots/Ankle/Caterpillar/107452.2754.jpg
Boots/Ankle/Caterpillar/107456.20.jpg
Boots/Ankle/Caterpillar/107456.69.jpg
Boots/Ankle/Caterpillar/107456.328.jpg
Boots/Ankle/Caterpi

Shoes/Oxfords/Rockport/114292.291.jpg
Shoes/Oxfords/Rockport/114292.360.jpg
Slippers/Slipper Flats/Old Friend/114508.3208.jpg
Slippers/Slipper Flats/Old Friend/114510.6360.jpg
Slippers/Slipper Flats/Old Friend/114511.8670.jpg
Shoes/Flats/Propet/114556.72.jpg
Sandals/Flat/Birkenstock/114640.2363.jpg
Sandals/Flat/Birkenstock/114640.66308.jpg
Sandals/Flat/Birkenstock/114640.364594.jpg
Sandals/Flat/Birkenstock/114640.364596.jpg
Sandals/Flat/Birkenstock/114687.223718.jpg
Sandals/Flat/Papillio/114688.368103.jpg
Shoes/Oxfords/Florsheim/114829.72.jpg
Shoes/Sneakers and Athletic Shoes/Timberland PRO/115074.6370.jpg
Boots/Ankle/Timberland PRO/115075.2099.jpg
Boots/Ankle/Timberland PRO/115075.22049.jpg
Shoes/Sneakers and Athletic Shoes/adidas/115191.151.jpg
Shoes/Sneakers and Athletic Shoes/adidas/115191.7968.jpg
Sandals/Athletic/adidas/115220.151.jpg
Sandals/Athletic/adidas/115220.20569.jpg
Sandals/Flat/adidas/115220.124318.jpg
Boots/Mid-Calf/UGG/115328.3.jpg
Boots/Mid-Calf/UGG/115328.9.jpg
Boot

Boots/Ankle/Lugz/125632.368074.jpg
Shoes/Sneakers and Athletic Shoes/Vans Kids/125668.3.jpg
Shoes/Sneakers and Athletic Shoes/Vans Kids/125668.9.jpg
Shoes/Sneakers and Athletic Shoes/Vans Kids/125668.585.jpg
Shoes/Sneakers and Athletic Shoes/Vans Kids/125668.24154.jpg
Shoes/Sneakers and Athletic Shoes/Vans Kids/125668.38877.jpg
Shoes/Sneakers and Athletic Shoes/Vans Kids/125668.183092.jpg
Shoes/Sneakers and Athletic Shoes/Vans Kids/125668.293798.jpg
Shoes/Sneakers and Athletic Shoes/Stride Rite/126018.3.jpg
Shoes/Sneakers and Athletic Shoes/Stride Rite/126018.15.jpg
Shoes/Sneakers and Athletic Shoes/Stride Rite/126019.15.jpg
Shoes/Sneakers and Athletic Shoes/Stride Rite/126019.72.jpg
Boots/Mid-Calf/Kamik Kids/126235.3.jpg
Boots/Mid-Calf/Kamik Kids/126235.9.jpg
Boots/Mid-Calf/Kamik Kids/126235.275.jpg
Boots/Mid-Calf/Kamik Kids/126235.113300.jpg
Boots/Mid-Calf/Kamik Kids/126235.258352.jpg
Shoes/Loafers/Dockers/126612.69.jpg
Shoes/Loafers/Dockers/126612.3322.jpg
Shoes/Sneakers and Athleti

Shoes/Sneakers and Athletic Shoes/New Balance Classics/7136102.9.jpg
Shoes/Sneakers and Athletic Shoes/New Balance Classics/7136102.85305.jpg
Shoes/Flats/Capezio/7136198.20210.jpg
Boots/Ankle/Timberland/7136724.20459.jpg
Shoes/Flats/Bloch/7136732.14.jpg
Shoes/Sneakers and Athletic Shoes/Bloch/7136735.20.jpg
Shoes/Sneakers and Athletic Shoes/Bloch/7136738.3.jpg
Shoes/Sneakers and Athletic Shoes/Bloch/7136746.3.jpg
Shoes/Sneakers and Athletic Shoes/Bloch/7136746.559.jpg
Sandals/Flat/Donald J Pliner/7137919.385.jpg
Sandals/Flat/Donald J Pliner/7137919.632.jpg
Shoes/Heels/Chinese Laundry/7138146.20777.jpg
Boots/Mid-Calf/Ariat/7138204.16158.jpg
Boots/Mid-Calf/Ariat/7138208.3.jpg
Boots/Mid-Calf/Ariat/7138208.16158.jpg
Boots/Mid-Calf/Ariat/7138208.250403.jpg
Boots/Mid-Calf/Ariat/7138210.10475.jpg
Boots/Mid-Calf/Ariat/7138210.206790.jpg
Boots/Mid-Calf/Ariat/7138210.224373.jpg
Boots/Mid-Calf/Ariat/7138210.361533.jpg
Boots/Ankle/La Sportiva/7138241.19.jpg
Boots/Mid-Calf/La Sportiva/7138243.3.jpg

...

Shoes/Flats/FootMates/7415279.91695.jpg
Shoes/Sneakers and Athletic Shoes/Aetrex/7415365.158.jpg
Shoes/Sneakers and Athletic Shoes/Aetrex/7415368.3.jpg
Shoes/Sneakers and Athletic Shoes/Aetrex/7415368.14.jpg
Shoes/Flats/Aetrex/7415388.3.jpg
Shoes/Sneakers and Athletic Shoes/Osiris/7415747.373270.jpg
Shoes/Sneakers and Athletic Shoes/Osiris/7415747.373271.jpg
Shoes/Oxfords/Frye/7416583.9448.jpg
Shoes/Oxfords/Frye/7416583.332550.jpg
Shoes/Oxfords/Frye/7416583.353336.jpg
Boots/Mid-Calf/Frye/7416587.195479.jpg
Boots/Mid-Calf/Frye/7416587.256487.jpg
Boots/Mid-Calf/Frye/7416768.1109.jpg
Boots/Mid-Calf/Frye/7416768.260274.jpg
Boots/Mid-Calf/Frye/7416768.301562.jpg
Boots/Ankle/Frye/7416773.301588.jpg
Boots/Ankle/Frye/7416773.353345.jpg
Boots/Mid-Calf/Frye/7416794.69259.jpg
Boots/Mid-Calf/Frye/7416794.263081.jpg
Boots/Mid-Calf/Frye/7416794.263084.jpg
Boots/Mid-Calf/Frye/7416799.301558.jpg
Boots/Mid-Calf/Frye/7416799.301571.jpg
Shoes/Flats/Circa Joan & David/7416956.2007.jpg
Shoes/Sneakers and A

Shoes/Heels/LifeStride/7424156.9.jpg
Shoes/Heels/LifeStride/7424156.154856.jpg
Shoes/Heels/LifeStride/7424156.155092.jpg
Boots/Ankle/Timberland Kids/7424416.229.jpg
Boots/Ankle/Chippewa/7424478.6.jpg
Boots/Mid-Calf/Chippewa/7424479.6.jpg
Boots/Ankle/Giorgio Brutini/7424663.72.jpg
Shoes/Loafers/Minnetonka/7424668.155896.jpg
Boots/Ankle/Timberland Kids/7424844.229.jpg
Boots/Ankle/Timberland Kids/7424845.229.jpg
Boots/Ankle/Timberland Kids/7424960.13434.jpg
Shoes/Boat Shoes/Sperry Kids/7425142.6.jpg
Shoes/Boat Shoes/Sperry Kids/7425142.3575.jpg
Shoes/Oxfords/Sperry Kids/7425159.155510.jpg
Shoes/Sneakers and Athletic Shoes/Superga Kids/7425484.559.jpg
Shoes/Sneakers and Athletic Shoes/Superga Kids/7425484.2034.jpg
Shoes/Sneakers and Athletic Shoes/Superga Kids/7425484.11865.jpg
Shoes/Sneakers and Athletic Shoes/Superga Kids/7425484.21134.jpg
Boots/Knee High/La Canadienne/7425701.106.jpg
Boots/Knee High/La Canadienne/7425701.1567.jpg
Boots/Knee High/La Canadienne/7425701.363052.jpg
Shoes/Fl

Shoes/Flats/MICHAEL Michael Kors/7439207.72.jpg
Slippers/Slipper Flats/Acorn/7439248.120328.jpg
Slippers/Slipper Flats/Acorn/7439248.250386.jpg
Slippers/Slipper Flats/Acorn/7439248.366696.jpg
Slippers/Slipper Flats/Acorn/7439248.366713.jpg
Slippers/Slipper Flats/Acorn/7439249.3.jpg
Slippers/Slipper Flats/Acorn/7439249.275.jpg
Slippers/Slipper Flats/Acorn/7439249.250403.jpg
Slippers/Slipper Flats/Acorn/7439249.306611.jpg
Shoes/Sneakers and Athletic Shoes/Superga Kids/7439342.9.jpg
Shoes/Sneakers and Athletic Shoes/Superga Kids/7439342.14.jpg
Shoes/Sneakers and Athletic Shoes/Superga Kids/7439342.30799.jpg
Shoes/Sneakers and Athletic Shoes/Superga Kids/7439342.35250.jpg
Shoes/Sneakers and Athletic Shoes/Superga Kids/7439342.262459.jpg
Boots/Mid-Calf/Tundra Kids Boots/7439559.3.jpg
Boots/Knee High/Tundra Kids Boots/7439561.3.jpg
Slippers/Slipper Flats/Foamtreads Kids/7439614.34137.jpg
Slippers/Slipper Flats/Foamtreads Kids/7439615.3395.jpg
Shoes/Flats/Mia Kids/7440275.161715.jpg
Shoes/Sne

Shoes/Sneakers and Athletic Shoes/Merrell/7459655.336.jpg
Shoes/Sneakers and Athletic Shoes/Merrell/7459655.123538.jpg
Boots/Ankle/Merrell/7459661.714.jpg
Boots/Ankle/Merrell/7459667.11.jpg
Shoes/Sneakers and Athletic Shoes/Merrell/7459704.354.jpg
Shoes/Sneakers and Athletic Shoes/Merrell/7459704.714.jpg
Shoes/Sneakers and Athletic Shoes/Merrell/7459704.123071.jpg
Shoes/Sneakers and Athletic Shoes/Merrell/7459704.169033.jpg
Shoes/Sneakers and Athletic Shoes/Merrell/7459704.318867.jpg
Shoes/Sneakers and Athletic Shoes/Merrell/7459706.11.jpg
Shoes/Sneakers and Athletic Shoes/Merrell/7459706.1162.jpg
Shoes/Sneakers and Athletic Shoes/Merrell/7459706.169034.jpg
Shoes/Sneakers and Athletic Shoes/Merrell/7459724.529.jpg
Shoes/Sneakers and Athletic Shoes/Merrell/7459724.76069.jpg
Shoes/Sneakers and Athletic Shoes/Merrell/7459724.169037.jpg
Shoes/Sneakers and Athletic Shoes/Merrell/7459724.243588.jpg
Shoes/Sneakers and Athletic Shoes/Merrell/7459724.319011.jpg
Shoes/Sneakers and Athletic Shoes

Shoes/Sneakers and Athletic Shoes/O'Neill/7481182.3.jpg
Shoes/Heels/Stuart Weitzman/7483015.126729.jpg
Shoes/Heels/Stuart Weitzman/7483015.175241.jpg
Boots/Ankle/Salomon/7483056.174483.jpg
Boots/Ankle/Salomon/7483056.239052.jpg
Boots/Ankle/Salomon/7483056.330210.jpg
Boots/Ankle/Salomon/7483071.269525.jpg
Shoes/Heels/Calvin Klein/7483407.71.jpg
Boots/Ankle/Polo Ralph Lauren/7483772.716.jpg
Boots/Ankle/Polo Ralph Lauren/7483773.3759.jpg
Boots/Ankle/Polo Ralph Lauren/7483773.25309.jpg
Shoes/Heels/Kate Spade New York/7484535.933.jpg


In [ ]:
# #from utils import train_val_generator
# data_path = './data/' # path of the data
# train_path = './train'
# test_path = './test'

# TARGET_SZ = 224
# # I have generators, rather natn needing to loop thriuh files....
# input_shape = (TARGET_SZ,TARGET_SZ,3) #(img_width,i mg_height,img_channel)')

# def apply_to_images_in_subdirs(parent_dir, func, num_per_cls=None, **kwargs):
#     results = []
#     for cls_dir_name in listdir(parent_dir):
#         cls_dir = abspath(join(parent_dir, cls_dir_name))
#         r = _apply_to_first_n_in_dir(func, cls_dir, num_per_cls, **kwargs)
#         results += r
#     return results


# def _apply_to_first_n_in_dir(func, dir_, num_per_cls, **kwargs):
#     if not isdir(dir_):
#         return []
#     results = []
#     for path in listdir(dir_)[:num_per_cls]:
#         abspath_ = abspath(join(dir_, path))
#         result = func(abspath_, **kwargs)
#         results.append(result)
#     return results

# def load_img_arr(p):
#     return img_to_array(load_img(p))


# def train_val_generator(batch_size, train_path,test_path,target_size=(TARGET_SZ,TARGET_SZ)):
#     # creates generators 
#     gen_tr = _train_generator()
#     gen_val = _val_generator()

#     sample = apply_to_images_in_subdirs(train_path,load_img_arr,num_per_cls = batch_size)
#     sample = np.array(sample)
#     gen_tr.fit(sample)
#     gen_val.fit(sample)

#     gen_tr = init_directory_generator(gen_tr,train_path,batch_size,target_size=target_size)
#     gen_val = init_directory_generator(gen_val,test_path,batch_size,target_size=target_size)

#     return gen_tr,gen_val

# def _train_generator():
#     return ImageDataGenerator(
#         featurewise_center=True,
#         featurewise_std_normalization=True,
#         rotation_range=180,
#         zoom_range=0.2,
#         width_shift_range=0.2,
#         height_shift_range=0.2,
#         shear_range=0.3,
#         horizontal_flip=True,
#         vertical_flip=True,
#         fill_mode='reflect'
#     )

# def _val_generator():
#     return ImageDataGenerator(
#         featurewise_center=True,
#         featurewise_std_normalization=True
#     )



# def init_directory_generator(gen, dir_,batch_size, target_size=(TARGET_SZ,TARGET_SZ),
#                              class_mode='categorical',shuffle=True):

#     return gen.flow_from_directory(
#         dir_,
#         class_mode = class_mode,
#         batch_size = batch_size,
#         target_size = target_size,
#         shuffle = shuffle
#     )

# batch_size = 32

# # train and test generator
# train_gen, val_gen = train_val_generator(batch_size, train_path, test_path)


# # I can reuse these later when we are actually fitting things.   

# # first lets just get the model vectors for each images...


In [ ]:
## train and test generator
#train_gen, val_gen = train_val_generator(batch_size, train_path, test_path)

In [ ]:
df

# for index, layer in enumerate(tr_model.layers):
#     print(index, layer.name)

In [ ]:
#Loop over paths to get updated database
imagepath = './data/'
df_features = add_features_to_database(summary_file, imagepath, cnn_model)

In [ ]:


from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import RidgeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
%matplotlib inline

#Start assessing models. Try a family of models to identify the best performers 






#Combine Data, get test/train split 

all_dataframes = pd.concat([painting_df, frame_df, difficult_df]).reset_index(drop=True)
labels = pd.DataFrame(all_dataframes['labels'])
image_features = pd.DataFrame((all_dataframes['image_features']))

X_train_pd, X_test_pd, Y_train_pd, Y_test_pd = train_test_split(image_features, labels, test_size = 0.3, random_state = 0)

X_train = np.vstack(X_train_pd['image_features'])
X_test = np.vstack(X_test_pd['image_features'])
Y_train = Y_train_pd.values.flatten()
Y_test = Y_test_pd.values.flatten()


#Instantiate diffent classifier types
clf_logit = LogisticRegression(solver='lbfgs', multi_class='ovr', max_iter = 1000)
ridge_model = RidgeClassifier(max_iter = 1000)

neighbors = [1,3,5]
knn_models = [KNeighborsClassifier(n_neighbors=num) for num in neighbors]

n_estimators = [10,100,500]
forest_model = RandomForestClassifier(n_estimators=500, )

models = [clf_logit, ridge_model, knn_models[0], knn_models[1], knn_models[2], forest_model]
model_names = ['logistic','ridge', 'knn_1', 'knn_3', 'knn_5', 'forest_model']


#Determine Model Scores
avg_scores = []
std_scores = []
for clf, name in zip(models, model_names): 
    print('training: ', name, ' model')
    scores = cross_val_score(clf, X_train, Y_train, cv = 5)
    avg_scores.append(np.mean(scores))
    std_scores.append(np.std(scores))
    
   

In [ ]:
sv = _45

In [ ]:
df = df.merge(df_features, left_index=True, right_index=True)

# this is where our cleaned up file will put put
summary_file = f'./summary_withFeatures_{today:%b-%d-%Y}.pkl'
print(summary_file)

df.to_pickle(summary_file)

In [ ]:
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import random 
from PIL import Image
import requests
from io import BytesIO

def nearest_neighbor_image_finder(query_image, query_features, num_neighs, database, fig_plot): 
    ''' function to return nearest neighbors for a query
    
    Args: 
        query_image (image): image 
        query_features (array): numpy array of extracted image features
        num_neighs (int): number of neighbors to return 
        datbase(df): pandas dataframe containing image database
        fig_plot (bool): boolean to indicate of plotting is desired (1 for yes)
        #min_price (float): minimum budget
        #max_price (float): max budget
        #remove_dup (bool): remove image from database if it exists
    
    return: 
        nn_results (arr): indices of nearest_neighbor lookup
        images_urls (arr): array of image urls for query and nearest neighbors sorted in nearest neighbor order
        pandas_list (pd): pandas dataframe containing all relevant information of neighbors 
    '''
    
    remove_dup = True
    
    #logic here in case the n exceeds number of items in database
    size_filter_frame = database.shape[0]
    num_neighs = min(num_neighs, size_filter_frame)
    
    #Fit Nearest Neighbor Model
    database_features = np.vstack(database['image_features'])
    neighs = NearestNeighbors(n_neighbors=num_neighs+1) #add plus one in case image exists in database
    neighs.fit(database_features)
    distance, nn_index = neighs.kneighbors(query_features, return_distance=True)
    
    #Check to see if image is already in dabase and remove (see if distance == 0)
    #If True, remove n = 0, and take n+1 neighbor
    if remove_dup: 
        updated_nn_index = []
        for i,row in enumerate(distance):
            image_exists = len(nn_index[i, row!=0]) == num_neighs
            if image_exists: 
                new_in= nn_index[i,1:num_neighs+1]
            else: 
                new_in = nn_index[i,0:num_neighs]
        
            updated_nn_index.append(new_in)
    
        nn_index = np.array(updated_nn_index)
    
    #Get pandas dataframes
    panda_list = []
    for row in nn_index: 
        panda_list.append(price_filter_df.iloc[row])
    
    #Get nearest neighbor result urls 
    url_matrix = []
    for row in nn_index: 
        url_row = price_filter_df['image_url'].values[row]
        url_matrix.append(url_row)
    
    url_matrix = np.array(url_matrix)

    #concat query image array to result arrays 
    images_urls = np.concatenate([query_image, url_matrix], axis = 1)

    if fig_plot: 
        get_fig_plots(images_urls)

    return(nn_index, images_urls, panda_list)  
            
def get_fig_plots(image_urls):
    ''' function to plot matrix of image urls. 
        image_urls[:,0] should be the query image
        
    Args: 
        image_urls: matrix of image urls
    
    return: 
        null
        saves image file to directory
    '''
    nrow = image_urls.shape[0]
    ncol = image_urls.shape[1]
    flat_urls = image_urls.flatten()

    fig = plt.figure(figsize = (20, 20))

    for num,url in enumerate(flat_urls):
    
        response = requests.get(url)
        img = Image.open(BytesIO(response.content))
        img = img.resize((224,224), Image.ANTIALIAS)
        plt.subplot(nrow, ncol, num+1)
        plt.axis('off')
        plt.imshow(img);

        if num == 0: 
            plt.title('Query')
    
        if 0<num <ncol: 
            plt.title('Neighbor ' + str(num))
        
    plt.savefig('image_search.png')
        
    
            
#Test Nearest Neighbor lookup with random selection from database, plot results 
n_queries = 1
query_inds = []
random.seed(150)
[query_inds.append(random.randint(i,12000)) for i in range(n_queries)]

query_features = np.vstack(image_df['image_features'][query_inds])
query_url = np.array(image_df['image_url'][query_inds]).reshape(n_queries,1)
num_neigs = 5
fig_plot = 1
indices, urls, subset_pd = nearest_neighbor_image_finder(query_url, query_features, num_neighs, image_df,fig_plot)

In [ ]:
### NOW A FEW THINGS
# train the model to "classify" (for validation)
# use the pre-trained model to look at how the features are distributed in space from MobileNetV2

print(tr_model.summary())

In [ ]:
# model saving
checkpoint = ModelCheckpoint(model_name+'.h5',monitor='val_acc',verbose=1,save_best_only=True)
early_stop = EarlyStopping(monitor='val_acc',min_delta=0,patience=10,verbose=1,mode='auto')

# Compile the model
#adam = Adam()
tr_model.compile(loss='categorical_crossentropy', optimizer = "nadam" ,metrics=['accuracy'])

In [ ]:
# train the model
history = tr_model.fit_generator(
                train_gen,
                steps_per_epoch=1000,
                epochs=30,
                validation_data = val_gen,
                validation_steps = 250,
                callbacks = [checkpoint, early_stop] )

In [ ]:
# plot the results
plt.figure()
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model_accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train','test'])
plt.savefig(args.model_name+'.jpg')
#plt.show()

In [ ]:
def central_crop(image):
    shape = tf.shape(image)
    min_dim = tf.reduce_min([shape[0], shape[1]])
    top_crop = (shape[0] - min_dim) // 4
    bottom_crop = shape[0] - top_crop
    left_crop = (shape[1] - min_dim) // 4
    right_crop = shape[1] - left_crop
    return image[top_crop:bottom_crop, left_crop:right_crop]


def preprocess(image, label, randomize=False):
    if randomize:
        cropped_image = random_crop(image)
        cropped_image = tf.image.random_flip_left_right(cropped_image)
    else:
        cropped_image = central_crop(image)
    resized_image = tf.image.resize(cropped_image, [224, 224])
    final_image = keras.applications.xception.preprocess_input(resized_image)
    return final_image, label

batch_size = 32
train_set = train_set_raw.shuffle(1000).repeat()
train_set = train_set.map(partial(preprocess, randomize=True)).batch(batch_size).prefetch(1)
valid_set = valid_set_raw.map(preprocess).batch(batch_size).prefetch(1)
test_set = test_set_raw.map(preprocess).batch(batch_size).prefetch(1)

images_resized = tf.image.resize(images, [224, 224])
plot_color_image(images_resized[0])
plt.show()


def plot_image(image):
    plt.imshow(image, interpolation="nearest")
    plt.axis("off")
    
    
def preprocess(image, label):
    resized_image = tf.image.resize(image, [224, 224])
    final_image = keras.applications.xception.preprocess_input(resized_image)
    return final_image, label



test_split, valid_split, train_split = tfds.Split.TRAIN.subsplit([10, 15, 75])

test_set_raw = tfds.load("tf_flowers", split=test_split, as_supervised=True)
valid_set_raw = tfds.load("tf_flowers", split=valid_split, as_supervised=True)
train_set_raw = tfds.load("tf_flowers", split=train_split, as_supervised=True)



In [ ]:

train_df = pandas.read_csv("./train.csv")
valid_df = pandas.read_csv("./valid.csv")

train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_dataframe(
        dataframe=train_df,
        directory='data/train',
        x_col="filename",
        y_col="class",
        target_size=(150, 150),
        batch_size=32,
        class_mode='binary')

validation_generator = test_datagen.flow_from_dataframe(
        dataframe=valid_df,
        directory='data/validation',
        x_col="filename",
        y_col="class",
        target_size=(150, 150),
        batch_size=32,
        class_mode='binary')

model.fit_generator(
        train_generator,
        steps_per_epoch=2000,
        epochs=50,
        validation_data=validation_generator,
        validation_steps=800)


In [ ]:
data_folderpath = df_clean.path
#files = data_folderpath.to_list()
filepaths = data_folderpath.to_list()
len(filepaths)

In [ ]:
#filepaths = glob(glob_str)
batch_size = 1024
img_size = 224
n_files = len(filepaths)
cur_batch = 0

In [ ]:
while True:
    # drop last if it does not fit
    if (n_files - cur_batch*batch_size) < batch_size or cur_batch == 0:
        ids = np.random.randint(0, n_files, n_files)
        np.random.shuffle(ids)
        cur_batch = 0
    train_data = []
    for i in range(batch_size):
        image_ab = load_image(filepaths[ids[cur_batch*batch_size+i]])
        train_data.append([image_ab[:, :img_size], image_ab[:, img_size:]])

    cur_batch = (cur_batch + 1) % int(len(filepaths)/batch_size)

    train_data = np.array(train_data)
#    yield train_data, cur_batch



#data=ImageDataBunch.from_df(path="ut-zap50k-images-square/",df=df_clean, size=224,bs=64)

In [ ]:
data_folderpath = df_clean.path

files = data_folderpath.to_list()

In [ ]:
print(files[0:5])

In [ ]:
data_iterator = create_minibatches(data_folderpath.to_list, 128, 224)
#        data_folderpath + "/train/*.jpg", batch_size, image_size)

In [ ]:
#val_data_iterator = iterate_minibatches(
#         data_folderpath + "/val/*.jpg", n_checkpoint_samples, image_size)


img_ab_fixed, _ = next(data_iterator)
    

In [ ]:
data_iterator


#al_data_iterator = iterate_minibatches(data_folderpath + "/val/*.jpg", n_checkpoint_samples, image_size)

In [ ]:
from keras.applications.mobilenet_v2 import MobileNetV2
from keras.preprocessing import image
import numpy as np
from keras.applications.mobilenet_v2 import preprocess_input
from keras.applications import imagenet_utils

global pwidth
global pheight
global network_model

pwidth = 224
pheight = 224
    
global graph 
graph = tf.get_default_graph()
network_model = MobileNetV2(weights = 'imagenet', include_top = False, input_shape = (pwidth,pheight,3),pooling = 'avg')




img_data = image.load_img(img_path, target_size=(pwidth, pheight))
img_vector = image.img_to_array(img_data)
img_vector = np.expand_dims(img_vector, axis=0)
img_vector = preprocess_input(img_vector) #Problem here, must be convention of keras to pass by reference?
img_vector = imagenet_utils.preprocess_input(img_vector)


In [ ]:
img_features = network_model.predict(img_vector)

#Get pandas df of n clo
nn_index, neighbors_df = nearest_neighbor_image_finder(img_features, 100, database_df,0,100,100)
neighbors = neighbors_df


In [ ]:
def nearest_neighbor_image_finder(query_features, num_neighs, database, price_filt, min_price, max_price,): 
    ''' function to return nearest neighbors for a query
    
    Args: 
        query_features (array): numpy array of extracted image features
        num_neighs (int): number of neighbors to return 
        datbase(df): pandas dataframe containing image database
        fig_plot (bool): boolean to indicate of plotting is desired (1 for yes)
        min_price (float): minimum budget
        max_price (float): max budget
        remove_dup (bool): remove image from database if it exists
    
    return: 
        nn_results: indices of nearest_neighbor lookup
        neighbor_pd: pandas dataframe subsetted on returned values from knn search
    '''
    #########################################
    ''' i don't actually want nearest neighbors.... maybe an average or latent images and the text.
        should return text.
    '''
    # #############################################
    #Filter Database based on given price range
    if price_filt:
        database  = database[(database['price']< max_price) & (database['price']> min_price)]
    
    #logic here in case the n exceeds number of items in database
    size_filter_frame = database.shape[0]
    num_neighs = min(num_neighs, size_filter_frame)
    
    #Fit Nearest Neighbor Model
    database_features = np.vstack(database['image_features'])
    neighs = NearestNeighbors(n_neighbors=num_neighs) 
    neighs.fit(database_features)
    distance, nn_index = neighs.kneighbors(query_features, return_distance=True)

    #Get nearest neighbor subset
    neighbor_pd = []
    for row in nn_index: 
        neighbor_pd.append(database.iloc[row])


In [ ]:
# data_iterator = iterate_minibatches(data
#         data_folderpath + "/train/*.jpg", batch_size, image_size)
# val_data_iterator = iterate_minibatches(
#         data_folderpath + "/val/*.jpg", n_checkpoint_samples, image_size)

img_ab_fixed, _ = next(val_data_iterator)
# img_a_fixed, img_b_fixed = img_ab_fixed[:, 0], img_ab_fixed[:, 1]




    ones = np.ones((batch_size, ) + disc_patch, dtype=np.float32)
    zeros = np.zeros((batch_size, ) + disc_patch, dtype=np.float32)
    dummy = zeros

    for i in range(n_iterations):
        D.trainable = True
        G.trainable = False

        image_ab_batch, _ = next(data_iterator)
        loss_d = D_model.train_on_batch(
            [image_ab_batch[:, 0], image_ab_batch[:, 1]],
            [ones, zeros])

        D.trainable = False
        G.trainable = True
        image_ab_batch, _ = next(data_iterator)
        loss_g = G_model.train_on_batch(
            [image_ab_batch[:, 0], image_ab_batch[:, 1]],
            [ones, dummy])

        print("iter", i)
        if (i % iters_per_checkpoint) == 0:
            G.trainable = False
            fake_image = G.predict(img_a_fixed)
            log_images(fake_image, 'val_fake', i, logger)
            save_model(G, out_dir)

        log_losses(loss_d, loss_g, i, logger)

        

# #n = 12856 images
# # images are "path"

# # now need to split into test train for sci-kit learn

# from sklearn.model_selection import train_test_split


# X_train, X_test, y_train, y_test = train_test_split(
# ...     X, y, test_size=0.33, random_state=42)



X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)






In [ ]:


seed = 7
numpy.random.seed(seed)

dataframe = df_clean #pandas.read_csv("INPUTFILE.csv", skiprows=range(0, 0))

dataset = dataframe.values

In [ ]:
X = dataset[:,0:50].astype(float) # number of cols-1
Y = dataset[:,50]

encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)

encoded_Y = np_utils.to_categorical(encoded_Y)
print("encoded_Y=", encoded_Y) 
# baseline model
def create_baseline():
    # create model
    model = Sequential()
    model.add(Dense(5, input_dim=5, kernel_initializer='normal', activation='relu'))
    model.add(Dense(5, kernel_initializer='normal', activation='relu'))
    #model.add(Dense(2, kernel_initializer='normal', activation='sigmoid'))

    model.add(Dense(2, kernel_initializer='normal', activation='softmax'))

    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])  # for binayr classification
        #model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])  # for multi class
    return model


model=create_baseline();
history=model.fit(X, encoded_Y, batch_size=50, nb_epoch=500, validation_split = 0.2, verbose=1)

print(history.history.keys())
# list all data in history
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()


pre_cls=model.predict_classes(X)    
cm1 = confusion_matrix(encoder.transform(Y),pre_cls)
print('Confusion Matrix : \n')
print(cm1)


score, acc = model.evaluate(X,encoded_Y)
print('Test score:', score)
print('Test accuracy:', acc)